# Project: WEB ANALYTICS INSIGHTS

 ## Table of Contents

<ul>
    <li><a href="#intro">Introduction</a></li>
    <li><a href="#wrangling">Data Wrangling</a></li>
    <li><a href="#eda">Exploratory Data Analysis</a></li>
    <li><a href="#conclusions">Conclusions</a></li>
</ul>


<a id='intro'></a>
# Introduction

In 2013, the e-commerce website has had flactuating visits. We'd like to understand the probable cause of this situation and discover possible insights to be used develop a data-driven strategy to engage and retain site visitors. The dataset contains traffic records of an ecommerce website.


**Questions to help understand the dataset**
- What is the daily distribution of site visitors.
- What is the monthly distribution of site visitors.
- What is the average orders made by clients.
- What is the conversion rate?
- What is the bounce rate?



In [1]:
# Importing libraries

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings 

%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
# Load Data
df = pd.read_excel('Web_Analytics_Data.xlsx')
df.head()

,DAY,VISITS,ORDERS,HAS_PURCHASED_PRIOR,DEVICE,BOUNCES,ADD_TO_CART,PRODUCT_PAGE_VIEWS,SEARCH_PAGE_VIEWS,GENDER,AGE,INCOME,,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,2013-01-01,64340,2312,N,iPhone,21755,21501,41587,45582,F,24,451529,NaN,NaN,NaN,NaN
1,2013-01-02,63958,2427,N,iPhone,15675,21355,41392,45456,F,22,384768,NaN,NaN,NaN,NaN
2,2013-01-03,67390,2230,Y,iPhone,28199,17086,46559,51972,M,71,283793,NaN,NaN,No. of Males,2585.0
3,2013-01-04,58305,1814,N,iPhone,24380,17172,35612,41043,M,51,417355,NaN,NaN,NaN,NaN
4,2013-01-05,74434,2333,Y,iPhone,15518,19392,44692,55954,F,32,99205,NaN,NaN,No. of Females,2525.0


In [3]:
df.shape

(5110, 16)

<a id='wrangling'></a>
# Data Wrangling